[Aidan's notes are in square brackets]<br> [*Cuong's notes are italicized*]

[Very well-written with the use of functions!]<br>[*Thank you!*]

[ am I correct in saying we can look for changes in the time of flight but not the time of flight itself?
Unless someone made the recordings with a GPSDO and accurate time stamps and all. ]<br>
[*The answer will be discussed through email*]

# Sunrise Festival Analysis Notebook
This is the Jupyter Notebook for analyzing Sunrise Festival data (www.hamsci.org/sunrisefest). It was developed by Kristina Collins KD8OXT, based on work by Cuong Nguyen (ORCID 0000-0002-3769-7556) and other students at the University of Scranton.

### Overview
In this notebook, you'll compare your recorded signal to a template of the signal*. By finding the [cross-correlation](https://en.wikipedia.org/wiki/Cross-correlation) between parts of the template and your recording, you will identify the time at which you observed each part, and the time between parts. You will also look for evidence of multipath propagation and identify the delay between paths. After you submit your results, they will be combined with other submissions to look for the effects of sunrise on propogation around the world.

_\* The template signal is actually the same audio file used at the source transmitters WWV and WWVH! You can download and experiment with the files yourself at https://zenodo.org/record/5602094_

### How to Use This Notebook

*Note:* If you are running this notebook in Binder, your changes will not be saved. To make significant changes, you should download a local version.

1. At the top of the screen, under "Kernel," click "Restart and Clear Output."
2. Under "Cell," click "Run All." This may take some time to run. Verify that the notebook is able to run completely and successfully. 
3. Upload your own data file to Binder (click and drag into the folder structure at left). Change the filename and user input parameters below. Make sure you can hear the signal clearly when you submit the file.
4. Repeat Steps 1 and 2. There are parts of the notebook where you will have to customize the code according to your data file. These spots will be indicated by text that looks like this:

<div class="alert alert-danger">$\color{red}{\text{TODO}}$ Read the directions before you begin.</div>

5. Look at the results and make notes from your data.
6. At the top of the page, select "File > Download As > Notebook." 


[submit somewhere]<br>[*The answer will be discussed through email*]

Let's begin!

## User Input Parameters

<div class="alert alert-danger">Welcome! Input your file parameters here, then run the notebook.</div>

In [ ]:
# File path to the template signal
fname = "test.wav"

<div class="alert alert-danger">$\color{red}{\text{TODO}}$: Here's where you should input the filename of your audio or IQ file.</div>

<!--- In this case, we're using an IQ file recorded on a KiwiSDR belonging to Phil Karn, KA9Q. -->

In [ ]:
# File path to the collected signal
fname1 = "N6GN_20211115T190749_iq_15.wav"
# fname1 = "w2naf.com_2021-11-15T19_07_36Z_10000.00_iq.wav"
# fname1 = "N6GN_20211115T190749_am_15.wav"

<div class="alert alert-danger">$\color{red}{\text{TODO}}$: If your signal is an IQ file, set this to True. If it is an AM file, set this to False.</div>

In [ ]:
# Does the collected signal require AM modulation?
modulation = True

In [ ]:
# Do you want a custom sample rate as opposed to the value given by the wav files themselves?
# Leave fs_custom = -1 if the answer is NO
fs_custom = -1

<div class="alert alert-danger">$\color{red}{\text{TODO}}$: Let's log the date and physical location of the station where the data was collected. Note that the start time is embedded in the Kiwi filename: </div>

In [ ]:
lat = 0;
lon = 0;
# filestart = "2021-11-15T19:07:36"
filestart = "2022-11-15T19:07:49"

## Import Useful Libraries

First, we'll make sure the requisite packages are installed.

If you are running this on Binder, the packages will be installed automatically.

If you are not running on Binder, install the Python packages from `requirements.txt` (i.e. `pip install -r requirements.txt`).

Next, we'll import the packages we need.

[ matplotlib has an interactive version, so we shouldn't need to import plotly in addition. However, on my computer, the matplotlib version was hanging up. ]<br>
[*This works so I wouldn't bother changing it. Plus, there seems to be more additional steps to show matplotlib interactive plots, while plotly only requires one additional command in the command line.*]

[ some of these can go ]

In [ ]:
from os.path import splitext
import gc
import math
import json
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.transforms as transforms

import plotly.graph_objects as go

import scipy.signal
from scipy import signal
from scipy.io import wavfile

from IPython.display import Audio, display

import datetime

from sunrisefest_module import *

In [ ]:
# Matplotlib settings to make the plots look a little nicer.

%matplotlib inline

plt.rcParams['font.size']      = 18
plt.rcParams['font.weight']    = 'bold'
plt.rcParams['axes.grid']      = True
plt.rcParams['axes.xmargin']   = 0
plt.rcParams['grid.linestyle'] = ':'
plt.rcParams['figure.figsize'] = (10,6)

## Define Useful Functions
Next, we'll define a few functions that will be useful later on in the analysis:

In [ ]:
# create an empty array to hold the results we find
results = []

In [ ]:
def record(name: str, description: str, value, unit: str):
    '''Record a result of our analysis'''
    
    print(description.format(value))
    
    results.append({
        "name": name,
        "description": description,
        "value": value,
        "unit": unit})

In [ ]:
def save_results():
    '''Save the results to a file'''
    
    output_file = splitext(fname1)[0] + "-results.json"
    print(f"writing to file {output_file}")
    
    with open(output_file, 'w') as file:
        json.dump({
            "fname1": fname1,
            "fs_custom": fs_custom,
            "modulation": modulation,
            "original sample rate": fs_wav1,
            "upsampled sample rate": fs_upsampled_wav1,
            "latitude": lat,
            "longitude": lon,
            "results": results
        }, file, indent=2)

## Load manufactured signal

Before we load your recording, we'll load the template signal and extract a few key parts of the template. We'll save each of these parts into variables so that we can compare them to your recording later.

In [ ]:
# Load in the file and detect the sampling frequency
fs_wav0, wav0 = wavfile.read(fname)

# Create a time vector
t_wav0 = np.arange(len(wav0)) * (1./fs_wav0)

In [ ]:
# Using the sounddevice library, we can hear what the template sounds like if played as
play(wav0, fs_wav0)

In [ ]:
# Normalize the data
wav0_n = norm(wav0)
wav0_n

[ fix divide by zero errors ]

In [ ]:
plot_signal(t_wav0, wav0_n, title="Manufactured Signal")

### Extract white noise

The first part of the signal we are interested in is the burst of white noise. If second 0 is the start of the signal, the same 2 second burst is sent at seconds 10 - 12 and at seconds 37 - 39. Here, we'll grab the first one:

In [ ]:
# Extract the white noise
white_noise, t_white_noise = extract(wav0_n, 10, 12, fs_wav0)

In [ ]:
# # Plot the white noise
# plot_signal(t_white_noise, white_noise, title="Extracted White Noise")

In this experiment, we are interested in the time between the first and second burst of noise. We can find the time of each burst in the recording by finding the maximum cross-correlation between the ideal burst and the recording. We would then take the difference of these times to find the time between bursts.

To illustrate how this will work, let's cross-correlate the noise we extracted with the entire template signal. This simulates an ideal case where we could record the signal without propagation delay, interference, ambient noise, etc.

[ Is it more helpful to have this written out each time, or to make it a function ? ]

In [ ]:
R_test_white, t_test_white = crosscorrelate(wav0_n, white_noise, fs_wav0)

plot_correlation(t_test_white, R_test_white, title='Full Manufactured Signal vs. Manufactured White Noise')

The cross-correlation has two large peaks corresponding to the beginnings of each burst of white noise. Since this is an ideal case, the peaks occur at 10 and 37 seconds exactly, and we can subtract the two times to get a time of 27 seconds between bursts.

### Extract Chirps

The next part of the signal we'll look at is the chirp sounds. These occur in the template signal between 24 and 32 seconds.

In [ ]:
chirps, t_chirps = extract(wav0_n, 24, 32, fs_wav0)

In [ ]:
plot_signal(t_chirps, chirps, title="Extracted Chirps")

If we repeat the same cross-correlation procedure, we can find the start time of the chirps in the template signal.

In [ ]:
R_test_chirps, t_test_chirps = crosscorrelate(wav0_n, chirps, fs_wav0)

plot_correlation(t_test_chirps, R_test_chirps, title='Cross-Correlation between the Full Manufactured Signal and Manufactured Chirps')

Here, there are multiple peaks because the chirps repeat themselves. We are interested in the largest peak, which corresponds to the best match between the template and the signal.

The largest peak occurs at 24 seconds, which is the start time of the chirps as we expected.

## Load the Recorded Data and Perform AM Demodulation

Now that we have our templates, we will load your recording.

In [ ]:
# Note that IQ WAV files look like regular stereo WAV files, but instead of 
# the channels representing the left and right speakers, they represent the
# I and Q signals.

fs_wav1, iq = wavfile.read(fname1)
t_wav1      = np.arange(len(iq))*(1./fs_wav1)

print('Sample Rate: {!s} samples/sec'.format(fs_wav1))
if modulation:
    print('Number of Channels: {!s}'.format(iq.shape[1]))
print('Data Type: {!s}'.format(iq.dtype))

In [ ]:
iq_float = iq / (np.max(np.abs(iq))+1.0) 

In [ ]:
# If this signal requires AM demodulation, do that:
if modulation:
    wav1 = np.sqrt(iq_float[:,0]**2 + iq_float[:,1]**2)
else:
    wav1 = iq_float

Let's listen to the file we've imported:

In [ ]:
play(wav1, fs_wav1)

<div class="alert alert-danger">$\color{red}{\text{TODO}}$: Manually clip the signal to just the part we're interested in - starting a second or so before the test signal starts, ending about a minute later.</div>

[Explain what the user is supposed to do here.]

In [ ]:
# wav1 = extract(wav1, 50, 110, fs_wav1) # Extract at 50 and 110 seconds
# t_wav1      = np.arange(len(iq))*(1./fs_wav1) # Make sure time variable is consistent
play(wav1, fs_wav1)

In [ ]:
# Normalize the data
wav1_n = norm(wav1)

In [ ]:
plot_signal(t_wav1, wav1_n, title=fname1)

## Remove the DC Offset

[ explain ]

[ maybe a less confusing variable name? ]

In [ ]:
wav1_offset = dco(wav1)

In [ ]:
plot_signal(t_wav1, wav1_offset, title='DC-Removed '+fname1)

## Resample

[ explain ]

In [ ]:
new_number_of_samples = int(len(wav1_offset)/fs_wav1*fs_wav0)
print("Total number of samples in the collected signal: ",new_number_of_samples)

In [ ]:
fs_upsampled_wav1 = fs_wav0
print("New sampling rate of collected signal: {:f} samples per second".format(fs_upsampled_wav1))

In [ ]:
upsampled_wav1, t_upsampled_wav1 = signal.resample(wav1_offset, new_number_of_samples, t=t_wav1)

In [ ]:
plot_signal(t_upsampled_wav1, upsampled_wav1, title='Resampled Received Data')

## Timing of the Collected Chirps

Now we will implement the procedure we described earlier: cross-correlate the template with your recording and identify the time of best correlation. This procedure is implemented in the `find_timing_of` method.<br>

First, we can cross-correlate the template chirps with our entire recording:

In [ ]:
t_upsampled_chirps = find_timing_of(chirps, upsampled_wav1, fs_upsampled_wav1)
print("Start Time of the Chirps relative to the beginning of the recording: {:f} seconds".format(t_upsampled_chirps))

## Timing of the First White Gaussian Noise

We know that the first white noise burst happens before the chirps. So, we will restrict our search for the first white Gaussian noise from the beginning of the recording up to the timing of the chirps which was found in the previous step.

In [ ]:
wav1_extract_for_t1, t_wav1_extract_for_t1 = extract(upsampled_wav1, 'start', t_upsampled_chirps, fs_upsampled_wav1)
wav1_extract_for_t1

In [ ]:
t1 = find_timing_of(white_noise, wav1_extract_for_t1, fs_upsampled_wav1)

This variable, `t1`, is our first finding. We'll save it for later using the `record` helper function. (At the end of this notebook, we'll write everything saved with `record` to a file that you can submit along with your raw recording.)

In [ ]:
record('t1', 'Start time of the first white Gaussian noise with respect to the beginning of the recording: {:f} seconds', t1, 'seconds')

It is useful to find the timing of the chirps with respect to the timing of the first white noise burst, so we will save it for later as well.

In [ ]:
t_chirps_wrt_t1 = t_upsampled_chirps - t1
record(
    't_chirps_wrt_t1',
    "Start Time of the Chirps with respect to Start Time of the First Noise: {:f} seconds",
    t_chirps_wrt_t1,
    'seconds')
print("Start Time of the Chirps with respect to Start Time of the First Noise: {:f} seconds".format(t_upsampled_chirps + t1))

## Timing of the Second White Gaussian Noise

Now we'll find how long after the first white noise the second noise starts.<br>

Once again, we can restrict our search to after the timing of the chirps found in the first step.

In [ ]:
# Collected signal extracted without the portion before the end of the chirps (approximately
wav1_extract_for_t2, t_wav1_extract_for_t2  = extract(upsampled_wav1, t_upsampled_chirps, 'end', fs_upsampled_wav1)

In [ ]:
t2 = find_timing_of(white_noise, wav1_extract_for_t2, fs_upsampled_wav1) + t_chirps_wrt_t1

record(
    't2',
    "Start time of the second white Gaussian noise with respect to the first white Gaussian noise: {:f} seconds.",
    t2,
    "seconds")

We can also find the time of second burst relative to the beginning of the recording by adding the times `t1` and `t2`:

In [ ]:
print("Start time of the second white Gaussian noise with respect to the beginning of the recording: {:f} seconds.".format(t2 + t1))

## Look for multipath

Let's investigate the correlation plot between the collected signal and the template chirps, and save the plot to a file.

In [ ]:
R_chirps, tau_chirps = crosscorrelate(upsampled_wav1, chirps, fs_upsampled_wav1)

fig = plot_correlation(tau_chirps, R_chirps, title='Cross-Correlation between Template Chirps and Signal between Noises\nsignal: {}'.format(fname1), return_figure=True)

[ could we make a clearer illustration for the correlations ? ]

In [ ]:
fig.savefig(splitext(fname1)[0] + "-results.png", dpi=300, facecolor='white')

In addition to the five large peaks, you might (or might not) see smaller peaks that are delayed slightly. If you can see them, they are likely evidence that you received the signal along more than one propagation path.

<div class="alert alert-danger">Do you see multipath? (Set the variable to True if you do, False if you don't.)</div>

In [ ]:
multipath = False

In [ ]:
record('multipath', 'user saw multipath? {}', multipath, 'yes/no')

Let's make an interactive version of this plot so we can zoom in and identify the location of the peaks.

<div class="alert alert-danger"><p>Uncomment the following line of code to generate an interactive plot.</p>
    <p><i>Uncomment Python code by removing the '#' before a line.</i></p></div>

In [ ]:
R_peaks, t_peaks = plot_correlation_interactive(tau_chirps, R_chirps, title='Cross-correlation')

<div class="alert alert-danger"><p>Zoom in to the area with the five peaks. How many different propagation paths can you see?</p>
<p>Set the variable to the number of paths you see. Remember, a group of five large peaks by themselves means you heard only one path. If the large peaks are each followed by one smaller 'echo', you heard a second path. If there were two 'echos', you heard 3 paths, etc.</p></div>

In [ ]:
number_of_paths = 1

In [ ]:
record('number_of_paths', 'The user was able to identify {} paths', number_of_paths, 'count')

We'll next save the peaks that were found in the cross-correlation (the red plus marks).

In [ ]:
record('R_peaks', 'Array of amplitudes of peaks: {}', R_peaks, 'unitless (cross-correlation)')
record('t_peaks', 'Array of times of peaks: {}', t_peaks, 'seconds')

[ the peak finding method needs tuning. ]<br>
[*Work in progress*]

## Output results

Finally, write the results we've been saving with the `record` function to a file:

In [ ]:
save_results()

## The End

That's it! Along with your recording, submit both the `.json` file with your numerical results and the `.png` file with your plot of the chirp correlations.